In [1]:
import numpy as np
from scipy.stats import norm

First, a bit on notation.  We will denote the distribution functions as follows:<br>
$\phi = $ Normal probability density function <br>
$\Phi = $ normal cumulative density function

For more information on the normal distribution see https://en.wikipedia.org/wiki/Normal_distribution

We define the following symbols:
- $S$ = Stock Price<br>
- $K$ = Strike price<br>
- $t$ = Time to expiration (years)<br>
- $r$ = Risk-free rate<br>
- $\sigma$ = Implied Volatility

The Black Scholes model gives:
- Call Price = $S\Phi(d_1) -Ke^{-rt}\Phi(d_2)$ <br>
- Put Price = $-S\Phi(-d_1) + Ke^{-rt}\Phi(-d_2)$

In the above equations, $d_1$ and $d_2$ are given respectively by,
$$d_1 = \frac{1}{\sigma \sqrt{t}} \left[ \ln\left(\frac{S}{K}\right) + \left(r + \frac{\sigma.^2}{2}\right) t\right],$$
and
$$d_2 = d_1 - \sigma \sqrt{t}.$$

Functions that return the Black-Scholes price of an option and the values of $d_1$ and $d_2$ are given in the cell below.

In [2]:
#  Functions that return d_1, d_2 and call and put prices
def d(sigma, S, K, r, t):
    d1 = 1 / (sigma * np.sqrt(t)) * ( np.log(S/K) + (r + sigma**2/2) * t)
    d2 = d1 - sigma * np.sqrt(t)
    return d1, d2

def call_price(sigma, S, K, r, t, d1, d2):
    C = norm.cdf(d1) * S - norm.cdf(d2) * K * np.exp(-r * t)
    return C

def put_price(sigma, S, K, r, t, d1, d2):
    P = -norm.cdf(-d1) * S + norm.cdf(-d2) * K * np.exp(-r * t)
    return P

From the Wikipedia article, we get the expressions for $\Delta$, $\Gamma$, and $\Theta$:

$$\Delta_{\mbox{call}} = \Phi(d_1)$$
$$\Delta_{\mbox{put}} = \Phi(-d_1)$$

Gamma is the same for both calls and puts,
$$\Gamma = Ke^{-et} \frac{\phi(d_2)}{S^2\sigma\sqrt{t}} $$

For $\Theta$ we have,
$$\Theta_{\mbox{call}} = -\frac{S\sigma\phi(d_1)}{2\sqrt{t}} -rKe^{-rt}\Phi(d_2)$$
$$\Theta_{\mbox{put}} = -\frac{S\sigma\phi(-d_1)}{2\sqrt{t}} -rKe^{-rt}\Phi(-d_2)$$

In [3]:
#  Functions for Deltam Gamma, and  Theta
def delta(d_1, contract_type):
    if contract_type == 'c':
        return norm.cdf(d1)
    if contract_type == 'p':
        return -norm.cdf(-d_1)
    
def gamma(d2, S, K, sigma, r, t):
    return( K * np.exp(-r * t) * (norm.pdf(d2) / (S**2 * sigma * np.sqrt(t) ))) 

def theta(d1, d2, S, K, sigma, r, t, contract_type):
    if contract_type == 'c':
        theta = -S * sigma * norm.pdf(d1) / (2 * np.sqrt(t)) - r * K * np.exp(-r * t) * norm.cdf(d2)
    if contract_type == 'p':
        theta = -S * sigma * norm.pdf(-d1) / (2 * np.sqrt(t)) + r * K * np.exp(-r * t) * norm.cdf(-d2)

    return theta

def vega(sigma, S, K, r, t):
    d1, d2 = d(sigma, S, K, r, t)
    v = S * norm.pdf(d1) * np.sqrt(t)
    return v

In [9]:
#  Using TSLA data from 5/9/2020 closing values
stockprice = 14755.4;
strikeprice = 14850;
interestrates = 0.1;
daysToExpire = 0;
iv = 25.06;
S = stockprice; print('S = ', S)
K = strikeprice; print('K = ', K)
r = interestrates; print('r = ', r)
t = daysToExpire / 365; print('t = ', t)
sigma = iv; print('sigma = ', sigma)

S =  14755.4
K =  14850
r =  0.1
t =  0.0
sigma =  25.06


In [12]:
#  Calculate the values of d1 and d2 needed for other functions
d1, d2 = d(sigma, S, K, r, t)
print('d1 = ', d1)
print('d2 = ', d2)

d1 =  -inf
d2 =  -inf


c:\pythonproject\tradingpy\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
delta_call = delta(d1, 'c')
delta_put = delta(d1, 'p')
print('Call Delta = ', delta_call)
print('Put Delta = ', delta_put)

Call Delta =  0.0
Put Delta =  -1.0


In [7]:
print( 'Gamma = ', gamma(d2, S, K, sigma, r, t) )

Gamma =  1.667417548269687e-05


In [8]:
#####  Calculate Theta
print(S, K, r, t, sigma, d1, d2)
print( theta(d1, d2, S, K, sigma, r, t, 'c') )

print( 'Call Theta = ', theta(d1, d2, S, K, sigma, r, t, 'c') / 365 * 100)
print( 'Put Thata = ', theta(d1, d2, S, K, sigma, r, t, 'p') / 365 * 100)
print( vega(sigma, S, K, r, t) /100 )

14755.4 14850 0.1 0.0027397260273972603 25.06 0.6511869337675167 -0.6605134162435586
-1140308.551009601
Call Theta =  -312413.301646466
Put Thata =  -312006.56378169544
2.492496426433954


#  Discussion of Units
Option price $\rightarrow$ price per share<br>
S $\rightarrow$  price per share <br>
K $\rightarrow$  price per share <br>
r $\rightarrow$  per year<br>
t $\rightarrow$  Years<br>
$\sigma \rightarrow$  per $\sqrt{\mbox{year}}$

One contract = 100 shares

## Example with $\Theta$
Fromthe Black Scholes model we see that $\Theta$ has units of price per share per year,
$$\Theta == \frac{\partial V}{\partial t}\rightarrow \left( \frac{\mbox{price}}{\mbox{share}}\right)\left(\frac{1}{\mbox{years}}\right)$$

But the convention for most trading platforms is,
$$\left(\frac{\mbox{price}}{\mbox{contract}}\right)\left(\frac{1}{\mbox{day}}\right)$$

We convert to price per contract per day by coming up with a conversion factor.
$$\left( \frac{\mbox{price}}{\mbox{share}}\right)\left(\frac{1}{\mbox{years}}\right) \times
\left(\frac{100\mbox{shares}}{1\mbox{contract}}\right)\left(\frac{1\mbox{year}}{365\mbox{days}}\right)$$

Using AAPL as an example,
<IMG SRC="aapl_theta.png" WIDTH="40%">

## Example with vega ($\nu$)

Bega has units of,
$$  \mbox{vega} = \frac{\partial V}{\partial \sigma} \rightarrow \left(\frac{\mbox{price}}{\mbox{share}}\right)\left(\frac{1}{\mbox{vol point}}\right)$$

The platform quotes vega in units of price per share per vol percentage point.  To do the conversion we do the following,
$$  \mbox{vega} = \frac{\partial V}{\partial \sigma} \rightarrow \left(\frac{\mbox{price}}{\mbox{share}}\right)\left(\frac{1}{\mbox{vol point}}\right)
\left(\frac{1 \mbox{vol point}}{100\mbox{percentage points}}\right)$$
<IMG SRC="aapl_vega.png" WIDTH="40%">

2.807193915043112
